In [7]:
def close_window_os():
    cv2.destroyAllWindows()
    for i in range (1,5): #trick for closing the display window on macos
        cv2.waitKey(1)

In [8]:
#Problem 1
import os
import cv2
import pickle
import numpy as np

#Read image from the file
image = cv2.imread("vinh.jpg", 1)

facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

face_coordinates = facecascade.detectMultiScale(gray, 1.3, 4)

for (a, b, w, h) in face_coordinates:
    cv2.rectangle(image, (a, b), (a+w, b+h), (255, 0, 0), 2)
    
#Question 1:
print("Face detection's coordinate: ", face_coordinates)
cv2.imshow('frames', image)
cv2.waitKey(0)
close_window_os()

Face detection's coordinate:  [[547 189 298 298]]


In [13]:
import os
import cv2
import pickle
import numpy as np

face_data = []
i = 0

camera = cv2.VideoCapture(0)

facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

name = input('Enter your name --> ')
ret = True

# Face recognition using Knn
while(ret):
    ret, frame = camera.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face_coordinates = facecascade.detectMultiScale(gray, 1.3, 4)

        for (a, b, w, h) in face_coordinates:
            faces = frame[b:b+h, a:a+w, :]
            resized_faces = cv2.resize(faces, (50, 50))
            
            if i % 10 == 0 and len(face_data) < 10:
                face_data.append(resized_faces)
            cv2.rectangle(frame, (a, b), (a+w, b+h), (255, 0, 0), 2)
        i += 1

        cv2.imshow('frames', frame)

        if cv2.waitKey(1) == 27:
            break
    else:
        print('error')
        break
    

cv2.destroyAllWindows()
camera.release()
close_window_os()


face_data = np.asarray(face_data)
face_data = face_data.reshape(10, -1)

if 'names.pkl' not in os.listdir('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/'):
    names = [name]*10
    with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/names.pkl', 'wb') as file:
        pickle.dump(names, file)
else:
    with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/names.pkl', 'rb') as file:
        names = pickle.load(file)

    names = names + [name]*10
    with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/names.pkl', 'wb') as file:
        pickle.dump(names, file)


if 'faces.pkl' not in os.listdir('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/'):
    with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/faces.pkl', 'wb') as w:
        pickle.dump(face_data, w)
else:
    with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/faces.pkl', 'rb') as w:
        faces = pickle.load(w)

    faces = np.append(faces, face_data, axis=0)
    with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/faces.pkl', 'wb') as w:
        pickle.dump(faces, w)

Enter your name --> President Obama


In [9]:
#KNN for face identification

import cv2
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pickle

facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/faces.pkl', 'rb') as w:
    faces = pickle.load(w)

with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/names.pkl', 'rb') as file:
    labels = pickle.load(file)

camera = cv2.VideoCapture(0)

print('Shape of Faces matrix --> ', faces.shape)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(faces,labels)

# Face recognition using Knn
while True:
    ret, frame = camera.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_coordinates = facecascade.detectMultiScale(gray, 1.3, 5)

        for (a, b, w, h) in face_coordinates:
            fc = frame[b:b + h, a:a + w, :]
            r = cv2.resize(fc, (50, 50)).flatten().reshape(1,-1)
            text = knn.predict(r)
            cv2.putText(frame, text[0], (a, b-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(frame, (a, b), (a + w, b + w), (0, 0, 255), 2)

        cv2.imshow('livetime face recognition', frame)
        if cv2.waitKey(1) == 27:
            break
    else:
        print("error")
        break

cv2.destroyAllWindows()
camera.release()
close_window_os()

Shape of Faces matrix -->  (30, 7500)


ValueError: Found input variables with inconsistent numbers of samples: [30, 40]

In [6]:
print(labels)

['Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'Dr. Vinh Nguyen (AIO)', 'President Obama', 'President Obama', 'President Obama', 'President Obama', 'President Obama', 'President Obama', 'President Obama', 'President Obama', 'President Obama', 'President Obama', 'phuc', 'phuc', 'phuc', 'phuc', 'phuc', 'phuc', 'phuc', 'phuc', 'phuc', 'phuc']


In [18]:
#SVM for face identification

import cv2
from sklearn.svm import SVC
import numpy as np
import pickle

facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/faces.pkl', 'rb') as w:
    faces = pickle.load(w)

with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/names.pkl', 'rb') as file:
    labels = pickle.load(file)

camera = cv2.VideoCapture(0)

print('Shape of Faces matrix --> ', faces.shape)
svm = SVC(kernel="rbf")
svm.fit(faces, labels)

# Face recognition using Knn
while True:
    ret, frame = camera.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_coordinates = facecascade.detectMultiScale(gray, 1.3, 5)

        for (a, b, w, h) in face_coordinates:
            fc = frame[b:b + h, a:a + w, :]
            r = cv2.resize(fc, (50, 50)).flatten().reshape(1,-1)
            text = svm.predict(r)
            cv2.putText(frame, text[0], (a, b-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(frame, (a, b), (a + w, b + w), (0, 0, 255), 2)

        cv2.imshow('livetime face recognition', frame)
        if cv2.waitKey(1) == 27:
            break
    else:
        print("error")
        break

cv2.destroyAllWindows()
camera.release()
close_window_os()

Shape of Faces matrix -->  (20, 7500)


In [16]:
import cv2
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pickle

facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/faces.pkl', 'rb') as w:
    faces = pickle.load(w)

with open('/Users/nguyendinhvinh2004gmail.com/Desktop/New_Data/De Cuong mon học/Machine Learning Course - VinhND/Week 4/QSTC/Computer Vision Foundation/Computer Vision Foundation - Session 4 - FD/theory/dataset/names.pkl', 'rb') as file:
    labels = pickle.load(file)

camera = cv2.VideoCapture(0)


tree_clf = DecisionTreeClassifier(criterion='entropy',
                                  max_depth=2)
tree_clf.fit(faces, labels)

# Face recognition using Knn
while True:
    ret, frame = camera.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_coordinates = facecascade.detectMultiScale(gray, 1.3, 5)

        for (a, b, w, h) in face_coordinates:
            fc = frame[b:b + h, a:a + w, :]
            r = cv2.resize(fc, (50, 50)).flatten().reshape(1,-1)
            text = tree_clf.predict(r)
            cv2.putText(frame, text[0], (a, b-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(frame, (a, b), (a + w, b + w), (0, 0, 255), 2)

        cv2.imshow('livetime face recognition', frame)
        if cv2.waitKey(1) == 27:
            break
    else:
        print("error")
        break

cv2.destroyAllWindows()
camera.release()
close_window_os()